# Part 1
1. อ่านข้อมูลจาก Spreadsheet ที่ต้องการ (Load Data)
2. เตรียมข้อมูลให้พร้อมในการใช้งาน (Clean Data)
3. แสดงผลแบบ Pivot Table และ เขียนกลับไปเป็น Spreadsheet (Export Data)

### Set up Essential component

In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
from bokeh.io import show, output_notebook
from bokeh.models import (ColumnDataSource,
                          HoverTool,
                          LinearColorMapper,
                          BasicTicker,
                          PrintfTickFormatter,
                          ColorBar)
from bokeh.plotting import figure

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows', 100)

pd.set_option('display.max_rows', 100)

# Load the data
alternatively
```python
df_collections = pd.read_excel('~/Dropbox/425degree_Order/ลีสส่งของใหม่/ลิสส่งของ_ย้ายแล้วจ้า.xlsx', sheetname=None)
```

In [2]:
# ยอดขาย
df_collections = pd.read_excel('ยอดขาย_Sample.xlsx', sheetname='Sheet2')
df_collections = df_collections[df_collections['รหัสสินค้า'].str.contains('-') == True]
df_collections.set_index('รหัสสินค้า', inplace=True)
df_collections.rename(columns={df_collections.columns[0] : 'วันที่'}, inplace=True)
df_collections.sort_index(inplace=True)

In [3]:
# get จำนวนคงเหลือ
df_Stock = pd.read_excel('stock_Sample.xls', 'Stock_Update') # sheetname ชื่อ 'Stock_Update'
df_Stock = df_Stock[df_Stock['รหัสสินค้า'].str.contains('-') == True]
df_Stock.set_index('รหัสสินค้า', inplace=True)
df_Stock.sort_index(inplace=True)

In [5]:
# COGS and GP
file_Inventory_name = 'Inventory_Sample.xls'
# sheetname ชื่อ 'Inventory_Sample’ และเลือกอ่านบาง Column
df_Inventory = pd.read_excel(file_Inventory_name, sheetname=file_Inventory_name[:-4], parse_cols=[0,1,4,5,6,7])
df_Inventory = df_Inventory[df_Inventory['ค่าขายสินค้า'] > 0][df_Inventory['รหัสสินค้า'].str.contains('-') == True]
df_Inventory.set_index('รหัสสินค้า', inplace=True)
df_Inventory['%กำไร'] = df_Inventory['กำไรขั้นต้น'] / df_Inventory['ค่าขายสินค้า']
df_Inventory['ราคาขายเฉลี่ย'] = df_Inventory['ค่าขายสินค้า'] / df_Inventory['จำนวนที่ขาย']
df_Inventory['กำไรเฉลี่ย'] = (df_Inventory['ราคาขายเฉลี่ย'] * df_Inventory['%กำไร']).round(decimals=2)
df_Inventory.sort_index(inplace=True)

/Users/BachKukkik/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


## Clean Some Data
### by creating Mask and Drop Some Rows

In [7]:
df_collections = df_collections[pd.isnull(pd.to_numeric(df_collections['จำนวนหยิบของ'], errors='coerce')) == False]

In [8]:
# Add a new column 'SKU_8digits' for first 8 digit SKU
df_collections['SKU_8digits'] = df_collections.index.map(lambda x : x.rpartition('-')[0])
df_collections = df_collections.join(df_Inventory['กำไรเฉลี่ย'])
df_collections = df_collections.replace([np.inf, -np.inf], np.nan)
df_collections = df_collections.join(df_Stock['ราคาขาย A'])
df_collections.loc[:,('กำไรเฉลี่ย')][np.isnan(df_collections['กำไรเฉลี่ย'])] = df_collections['ราคาขาย A'] * 0.3
df_collections['กำไรXจำนวน'] = (df_collections['จำนวนหยิบของ'] * df_collections['กำไรเฉลี่ย'])

df_Inventory['SKU_8digits'] = df_Inventory.index.map(lambda x : x.rpartition('-')[0])
df_Stock['SKU_8digits'] = df_Stock.index.map(lambda x : x.rpartition('-')[0])

/Users/BachKukkik/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Actually perform pivot table
1. Choose what and how we show value : 
```python
values='จำนวนหยิบของ', aggfunc=np.sum
```
2. Choose how we index on rows
```python
index=['รหัสสินค้า']
```
3. Choose how we run the columns
```python
columns=df_collections.columns[0]
```

In [9]:
# จำนวนกล่อง
pivot_Order_box = df_collections.pivot_table(values='จำนวนหยิบของ', index=['รหัสสินค้า'],
                                    columns=df_collections.columns[0],
                                    aggfunc=sum)

In [10]:
pivot_Order_box.columns = pivot_Order_box.columns.map(lambda x : x.strftime("%d-%m-%y"))
pivot_Order_box['Grand Total'] = pivot_Order_box.sum(axis=1)
pivot_Order_box = pivot_Order_box.join(df_collections['รายละเอียด']).join(df_Stock['จำนวนคงเหลือ'])
pivot_Order_box['Sum7D'] = pivot_Order_box[pivot_Order_box.columns[-10:-3]].sum(axis=1)
pivot_Order_box = pivot_Order_box[~pivot_Order_box.index.duplicated()]

## Pivot order by profit
1. Choose what and how we show value : 
```python
values='กำไรXจำนวน', aggfunc=np.sum
```
2. Choose how we index on rows
```python
index=['รหัสสินค้า']
```
3. Choose how we run the columns
```python
columns=df_collections.columns[0]
```

In [12]:
# perform pivot table
pivot_Order_profit = df_collections.pivot_table(values='กำไรXจำนวน',
                                            index=['SKU_8digits'],
                                            columns=['วันที่'],
                                            aggfunc=np.sum)

In [13]:
pivot_Order_profit.columns = pivot_Order_profit.columns.map(lambda x : x.strftime("%d-%m-%y"))
pivot_Order_profit['Grand Total'] = pivot_Order_profit.sum(axis=1)
pivot_Order_profit = pivot_Order_profit.join(df_collections.reset_index().set_index('SKU_8digits')['รายละเอียด'])
pivot_Order_profit['GrossProfit7D'] = pivot_Order_profit[pivot_Order_profit.columns[-9:-2]].sum(axis=1)
pivot_Order_profit = pivot_Order_profit.join(df_Stock.pivot_table(values='จำนวนคงเหลือ',index=['SKU_8digits'],aggfunc=np.sum))
pivot_Order_profit = pivot_Order_profit[~pivot_Order_profit.index.duplicated()]

# Part 2
4. เขียน Function Search สินค้า (Filter Data)
5. Plot Graph ให้เห็นภาพรวม (Data Visualization)

# Useful commands for 425degree

In [15]:
def filter_Stock_any_cond(string):
    def filter_Stock_by_รายละเอียด(string):
        string = str(string).lower()
        return df_Stock[df_Stock['รายละเอียด'].str.contains(string, case=False, na=False)][df_Stock.columns[pd.np.r_[0,3,4]]][df_Stock['รายละเอียด'].str.contains('เลิก') == False]

    def filter_Stock_by_SKU(string):
        string = str(string).lower()
        return df_Stock[df_Stock.index.str.contains(string, case=False, na=False)][df_Stock.columns[pd.np.r_[0,3,4]]][df_Stock['รายละเอียด'].str.contains('เลิก') == False]
    return pd.concat([filter_Stock_by_รายละเอียด(string), filter_Stock_by_SKU(string)])


def order_table_any_cond(string):
    def order_table_รายละเอียด(string):
        string = str(string).lower()
        return pivot_Order_box[pivot_Order_box['รายละเอียด'].str.contains(string, case=False, na=False)][pivot_Order_box.columns[pd.np.r_[-3,-11:-4,-1,-2]]]
    
    def order_table_SKU(string):
        string = str(string).lower()
        return pivot_Order_box[pivot_Order_box.index.str.contains(string, case=False, na=False)][pivot_Order_box.columns[pd.np.r_[-3,-11:-4,-1,-2]]]
    return pd.concat([order_table_รายละเอียด(string), order_table_SKU(string)])


def profit_table_any_cond(string):
    def profit_table_รายละเอียด(string):
        string = str(string).lower()
        return pivot_Order_profit[pivot_Order_profit['รายละเอียด'].str.contains(string, case=False, na=False)][pivot_Order_profit.columns[pd.np.r_[-3,-11:-4,-2,-1]]].sort_values('GrossProfit7D', ascending=False)
    
    def profit_table_SKU(string):
        string = str(string).lower()
        return pivot_Order_profit[pivot_Order_profit.index.str.contains(string, case=False, na=False)][pivot_Order_profit.columns[pd.np.r_[-3,-11:-4,-2,-1]]].sort_values('GrossProfit7D', ascending=False)
    return pd.concat([profit_table_รายละเอียด(string), profit_table_SKU(string)])

# Data Visualization

In [16]:
data = pivot_Order_profit[pivot_Order_profit.columns[pd.np.r_[-214:-4]]]

In [17]:
data.columns.name = 'Date'
data.index.name = 'SKU'
SKU = list(data.index)
date = list(data.columns)

# reshape to 1D array or GProfit with SKU and date for each row.
df_plot = pd.DataFrame(data.stack(), columns=['GProfit']).reset_index().join(
    pivot_Order_profit['รายละเอียด'], on='SKU').reset_index().rename(columns={"รายละเอียด": 'detail'})

colors = ['#feeaea', 
'#fcdcdc', 
'#facdcd', 
'#f9bebe', 
'#fab1b1', 
'#faa3a3', 
'#f99292', 
'#f68080', 
'#f17070', 
'#ed6161', 
'#e54f4f', 
'#df4040', 
'#d83131', 
'#d12323', 
'#c71616', 
'#be0a0a', 
'#b20303', 
'#a40202', 
'#950101', 
'#870101', 
'#750101', 
'#670101', 
'#5c0101', 
'#4d0101', 
'#3d0101', ]

mapper = LinearColorMapper(palette=colors, low=0, high=5000)

source = ColumnDataSource(df_plot)

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title = '425 Profit by SKU (from {0} to {1})'.format(date[0], date[-1]),
           x_range = SKU, y_range=list(reversed(date)),
           x_axis_location = 'above', plot_width = len(data.index) * 12, plot_height = 1600,
           tools = TOOLS, toolbar_location="below")

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = '5pt'
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = 1

p.rect(x='SKU',y='Date',width=1, height=1,
       source=source,
       fill_color={'field': 'GProfit', 'transform':mapper},
       line_color=None)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size = '5pt',
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     label_standoff=6, border_line_color=None, location=(0, 0))

p.add_layout(color_bar, 'right')

p.select_one(HoverTool).tooltips = [
     ('SKU', '@SKU'),
     ('รายละเอียด', '@detail'),
     ('GProfit', '@GProfit{1,111.11} THB'),
     ('Date', '@Date'),
]

In [18]:
output_notebook()

show(p, notebook_handle=True)

Loading BokehJS ...

## Uncomment the code too see results
* 1 line at a time

In [24]:
# pivot_Order_profit[pivot_Order_profit.columns[pd.np.r_[-3,-11:-4,-2,-1]]].sort_values('GrossProfit7D', ascending=False)
# filter_Stock_any_cond('jp') # return SKU, รายละเอียด, ราคาขาย, Stock
order_table_any_cond('US317') # return pivot table ของยอดส่ง 7 วันท้าย
# profit_table_any_cond('iphone8') # return pivot table ของยอดกำไร 7 วันท้าย

,รายละเอียด,02-03-18,05-03-18,06-03-18,07-03-18,08-03-18,09-03-18,10-03-18,Sum7D,จำนวนคงเหลือ
รหัสสินค้า,,,,,,,,,,
US317-AL-A,UTJ Urbana iPhone 7 Plus - Navy,5.0,9.0,47.0,1.0,2.0,29.0,8.0,101.0,378


In [21]:
%cd ~/Desktop/
with pd.ExcelWriter('box_ย้ายแล้วจ้า.xlsx') as writer:
    pivot_Order_box.to_excel(writer, sheet_name='All')

/Users/BachKukkik/Desktop


In [22]:
%cd ~/Desktop/
with pd.ExcelWriter('profit_ย้ายแล้วจ้า.xlsx') as writer:
    pivot_Order_profit.to_excel(writer, sheet_name='All')

/Users/BachKukkik/Desktop
